In [19]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from plot_map import process_sensor_data, process_rainfall_data
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats

In [20]:
base_path = os.path.normpath(os.path.join('..', 'dataset'))
senser_path = os.path.normpath(os.path.join(base_path, 'sensor'))
rain_station_path = os.path.join(base_path, 'rain_station')

In [ ]:
housing_commercial_construction_area_df = pd.read_csv(os.path.join(base_path, "6-1-4 房屋營建工程面積－建照執照.csv"))[['年', '地區', '使用執照辦公/服務類總樓地板面積']]
rows_to_remove = [0, 1, 2, 10, 16, 23, 26,27,28,29, 30]
housing_commercial_construction_area_df = housing_commercial_construction_area_df.drop(rows_to_remove)

In [280]:
housing_commercial_construction_area_df

,年,地區,使用執照辦公/服務類總樓地板面積
3,2021,臺北市,307696
4,2021,新北市,286203
5,2021,基隆市,6335
6,2021,新竹市,38915
7,2021,桃園市,128952
8,2021,新竹縣,87083
9,2021,宜蘭縣,20383
11,2021,臺中市,132886
12,2021,苗栗縣,16266
13,2021,彰化縣,19819


In [281]:
maxdamage_commercial = pd.DataFrame(
    {
    "Country": ["High Income"],
    "Building-based Max Damage Structure (€/m², 2010)": [515],
    "Building-based Max Damage Content (€/m², 2010)": [515],
    "Building-based Total (€/m², 2010)": [1030],
    "Land-use-based Total (€/m², 2010)": [309],
    "Object-based Total (€/object, 2010)": [206007]
}
)
maxdamage_commercial

,Country,"Building-based Max Damage Structure (€/m², 2010)","Building-based Max Damage Content (€/m², 2010)","Building-based Total (€/m², 2010)","Land-use-based Total (€/m², 2010)","Object-based Total (€/object, 2010)"
0,High Income,515,515,1030,309,206007


In [282]:
damage_function = pd.DataFrame(
    {
    "Damage class": ["Commercial buildings"] * 9,  # 全部都是 Commercial buildings
    "Flood depth [cm]": [0, 50, 100, 150, 200, 300, 400, 500, 600],
    "Damage function (ASIA)": [0.00, 0.38, 0.54, 0.66, 0.76, 0.88, 0.94, 0.98, 1.00],
    "Standard deviation (ASIA)": [0.00, 0.24, 0.24, 0.24, 0.25, 0.17, 0.11, 0.05, 0.00],
}
)

# 新的淹水深度範圍，每 10 cm 為一個間隔 (0 ~ 600 cm)
new_depths = np.arange(0, 601, 10)

# 利用 np.interp 進行線性插值
new_damage_values = np.interp(new_depths,
                              damage_function["Flood depth [cm]"],
                              damage_function["Damage function (ASIA)"])

new_std_values = np.interp(new_depths,
                           damage_function["Flood depth [cm]"],
                           damage_function["Standard deviation (ASIA)"])

# 建立新的 damage function DataFrame
damage_function = pd.DataFrame({
    "Damage class": ["Commercial buildings"] * len(new_depths),
    "Flood depth [cm]": new_depths,
    "Damage function (ASIA)": new_damage_values,
    "Standard deviation (ASIA)": new_std_values,
})


damage_function.head(10)

,Damage class,Flood depth [cm],Damage function (ASIA),Standard deviation (ASIA)
0,Commercial buildings,0,0.000,0.000
1,Commercial buildings,10,0.076,0.048
2,Commercial buildings,20,0.152,0.096
3,Commercial buildings,30,0.228,0.144
4,Commercial buildings,40,0.304,0.192
5,Commercial buildings,50,0.380,0.240
6,Commercial buildings,60,0.412,0.240
7,Commercial buildings,70,0.444,0.240
8,Commercial buildings,80,0.476,0.240
9,Commercial buildings,90,0.508,0.240


In [283]:
years_to_process = [2020, 2021, 2022, 2023, 2024, 2025]
year = years_to_process[4]

sensor_df_list = []
rain_station_df_list = []

for year in years_to_process:
    sensor_df = pd.read_csv(os.path.join(senser_path, f"flood_data_{year}_merged_filled.csv")).dropna()
    if year == 2024:
        # 找出 'value' 欄位中最大的兩個值的索引
        largest_two_idx = sensor_df['value'].nlargest(2).index
        # 移除這兩筆資料
        sensor_df = sensor_df.drop(largest_two_idx)
    
    sensor_df = sensor_df.rename(columns={'file_date': 'datetime'})
    sensor_df['datetime'] = pd.to_datetime(sensor_df['datetime'])
    sensor_df_list.append(sensor_df)
    
    rain_station_df = pd.read_csv(os.path.join(rain_station_path, f"rain_station_data_{year}_merged.csv")).dropna()
    rain_station_df_list.append(rain_station_df)
    
sensor_df = pd.concat(sensor_df_list, ignore_index=True)
rain_station_df= pd.concat(rain_station_df_list, ignore_index=True)

In [284]:
rain_station_df.head()

,StationId,datetime,CountyName,TownName,Past24hr,StationName
0,00H710,2020-01-01,南投縣,集集鎮,20.333333,NaN
1,81U900,2020-01-01,宜蘭縣,頭城鎮,4.810345,NaN
2,81U910,2020-01-01,宜蘭縣,蘇澳鎮,10.565517,NaN
3,O1D570,2020-01-01,新竹縣,寶山鄉,0.000000,NaN
4,81T860,2020-01-01,花蓮縣,鳳林鎮,1.537931,NaN


In [285]:
town_to_county_map = dict(zip(rain_station_df['TownName'], rain_station_df['CountyName']))
sensor_df['COUNTYNAME'] = sensor_df['TOWNNAME'].map(town_to_county_map)

In [286]:
station_count_dict, avg_depth_dict = process_sensor_data(sensor_df)
avg_rainfall_dict = process_rainfall_data(rain_station_df)

處理了 525436 筆淹水感測讀數
發現 138 個鄉鎮市的資料

感測站數量前5的鄉鎮市:
TOWNNAME
口湖鄉    22
宜蘭市    18
田中鎮    15
東石鄉    12
太保市    12
Name: station_id, dtype: int64

平均淹水深度前5的鄉鎮市:
TOWNNAME
八里區    1.253426
東勢鄉    1.087707
斗南鎮    0.790072
水林鄉    0.358695
鹽水區    0.183611
Name: value, dtype: float64
處理了 1888523 筆雨量資料
發現 355 個鄉鎮市的雨量資料

平均雨量前5的鄉鎮市:
TownName
暖暖區    12.334117
萬里區    12.327410
雙溪區    12.231254
平溪區    12.175949
瑞芳區    12.015910
Name: Past24hr, dtype: float64


In [287]:
# 將感測器數據按照日期和鄉鎮分組並計算平均淹水高度
sensor_grouped = sensor_df.groupby(['datetime', 'TOWNNAME', 'COUNTYNAME'])['value'].mean().reset_index()
sensor_grouped['datetime'] = pd.to_datetime(sensor_grouped['datetime'])
sensor_grouped.rename(columns={'value': 'flood_depth'}, inplace=True)
sensor_grouped.head()

,datetime,TOWNNAME,COUNTYNAME,flood_depth
0,2020-01-01,七股區,臺南市,0.0
1,2020-01-01,三峽區,新北市,0.0
2,2020-01-01,中和區,新北市,0.0
3,2020-01-01,中埔鄉,嘉義縣,0.0
4,2020-01-01,中正區,基隆市,0.0


In [288]:
# 將雨量資料按照日期和鄉鎮分組並計算平均雨量
rain_grouped = rain_station_df.groupby(['datetime', 'TownName', 'CountyName'])['Past24hr'].mean().reset_index()
rain_grouped['datetime'] = pd.to_datetime(rain_grouped['datetime'])
rain_grouped.rename(columns={'Past24hr': 'rainfall'}, inplace=True)
rain_grouped.head()

,datetime,TownName,CountyName,rainfall
0,2020-01-01,七堵區,基隆市,14.193273
1,2020-01-01,七股區,臺南市,0.577586
2,2020-01-01,三地門鄉,屏東縣,0.000000
3,2020-01-01,三峽區,新北市,2.893502
4,2020-01-01,三星鄉,宜蘭縣,4.155172


In [290]:
# 將感測器數據和雨量數據合併
# 使用鄉鎮名稱和日期作為合併鍵
# 考慮雨量延遲效應
sensor_grouped['matching_date'] = sensor_grouped['datetime'] # 感測器當天日期
rain_grouped['matching_date'] = rain_grouped['datetime'] + pd.Timedelta(days=1) # 雨量後一天日期

# 使用調整後的日期進行合併
merged_df = pd.merge(
    sensor_grouped,
    rain_grouped,
    left_on=['matching_date', 'TOWNNAME', 'COUNTYNAME'],
    right_on=['matching_date', 'TownName', 'CountyName'],
    how='inner'
)
merged_df = merged_df.merge(
    housing_commercial_construction_area_df[['地區', '使用執照辦公/服務類總樓地板面積']],
    left_on='CountyName',
    right_on='地區',
    how='left'
)

merged_df.head()

,datetime_x,TOWNNAME,COUNTYNAME,flood_depth,matching_date,datetime_y,TownName,CountyName,rainfall,地區,使用執照辦公/服務類總樓地板面積
0,2020-01-02,七股區,臺南市,0.0,2020-01-02,2020-01-01,七股區,臺南市,0.577586,臺南市,49423.0
1,2020-01-02,三峽區,新北市,0.0,2020-01-02,2020-01-01,三峽區,新北市,2.893502,新北市,286203.0
2,2020-01-02,中埔鄉,嘉義縣,0.0,2020-01-02,2020-01-01,中埔鄉,嘉義縣,1.794828,嘉義縣,11952.0
3,2020-01-02,中正區,基隆市,0.0,2020-01-02,2020-01-01,中正區,基隆市,1.414138,基隆市,6335.0
4,2020-01-02,二崙鄉,雲林縣,0.0,2020-01-02,2020-01-01,二崙鄉,雲林縣,0.334483,雲林縣,16316.0


In [ ]:
def categorize_flood_depth(depth_meters):
    """
    Categorize flood depth according to the ASIA damage function for commercial buildings.
    """
    # Convert depth from meters to centimeters
    depth_cm = depth_meters
    
    # Damage class thresholds in cm
    thresholds = damage_function['Flood depth [cm]']
    
    # Find the appropriate damage class
    for i in range(len(thresholds) - 1):
        if depth_cm >= thresholds[i] and depth_cm < thresholds[i + 1]:
            return i
    
    # If depth is greater than or equal to the highest threshold
    if depth_cm >= thresholds[-1]:
        return len(thresholds) - 1
    
    # Default to class 0 for depths below the first threshold
    return 0

def categorize_flood_damage(df):
    """
    Add damage class column to dataframe based on flood depth
    """
    result_df = df.copy()
    
    # Add damage class column
    result_df['damage_class'] = result_df['flood_depth'].apply(categorize_flood_depth)
    
    return result_df

categorized_df = categorize_flood_damage(merged_df)

damage_values = maxdamage_commercial['Land-use-based Total (€/m², 2010)'].values * damage_function['Damage function (ASIA)']
damage_mapping = {i: damage_values[i] for i in range(len(damage_values))}
categorized_df['damage_value'] = categorized_df['damage_class'].map(damage_mapping)
# 新增 total_damage 欄位：每一列的 damage_value * 建照執照商業類總樓地板面積
categorized_df['total_damage'] = categorized_df['damage_value'] * categorized_df['使用執照辦公/服務類總樓地板面積'].values
categorized_df = categorized_df.dropna()

categorized_df

,datetime_x,TOWNNAME,COUNTYNAME,flood_depth,matching_date,datetime_y,TownName,CountyName,rainfall,地區,使用執照辦公/服務類總樓地板面積,damage_class,damage_value,total_damage
0,2020-01-02,七股區,臺南市,0.0,2020-01-02,2020-01-01,七股區,臺南市,0.577586,臺南市,49423.0,0,0.0,0.0
1,2020-01-02,三峽區,新北市,0.0,2020-01-02,2020-01-01,三峽區,新北市,2.893502,新北市,286203.0,0,0.0,0.0
2,2020-01-02,中埔鄉,嘉義縣,0.0,2020-01-02,2020-01-01,中埔鄉,嘉義縣,1.794828,嘉義縣,11952.0,0,0.0,0.0
3,2020-01-02,中正區,基隆市,0.0,2020-01-02,2020-01-01,中正區,基隆市,1.414138,基隆市,6335.0,0,0.0,0.0
4,2020-01-02,二崙鄉,雲林縣,0.0,2020-01-02,2020-01-01,二崙鄉,雲林縣,0.334483,雲林縣,16316.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143350,2025-02-28,鳳林鎮,花蓮縣,0.0,2025-02-28,2025-02-27,鳳林鎮,花蓮縣,0.000000,花蓮縣,9712.0,0,0.0,0.0
143351,2025-02-28,鹽水區,臺南市,0.0,2025-02-28,2025-02-27,鹽水區,臺南市,0.000000,臺南市,49423.0,0,0.0,0.0
143352,2025-02-28,鹿港鎮,彰化縣,0.0,2025-02-28,2025-02-27,鹿港鎮,彰化縣,0.000000,彰化縣,19819.0,0,0.0,0.0
143353,2025-02-28,鹿草鄉,嘉義縣,0.0,2025-02-28,2025-02-27,鹿草鄉,嘉義縣,0.000000,嘉義縣,11952.0,0,0.0,0.0


In [313]:
year = 2020
flood_loss_path = os.path.join("..", "output", f"town_flood_loss_statistics_{year}.csv")
damage_df = pd.read_csv(flood_loss_path)

In [315]:
damage_df.groupby('TOWNNAME')['total_flood_loss'].mean().to_dict()

{'七股區': 0.0,
 '三峽區': 0.0,
 '中埔鄉': 0.0,
 '中正區': 0.0,
 '二崙鄉': 0.0,
 '二林鎮': 0.0,
 '二水鄉': 0.0,
 '五結鄉': 0.0,
 '五股區': 0.0,
 '仁德區': 0.0,
 '仁愛區': 0.0,
 '仁武區': 0.0,
 '伸港鄉': 0.0,
 '佳冬鄉': 0.0,
 '元長鄉': 1168.1858048780489,
 '光復鄉': 0.0,
 '內埔鄉': 0.0,
 '八德區': 0.0,
 '八里區': 0.0,
 '六腳鄉': 0.0,
 '冬山鄉': 0.0,
 '北區': 0.0,
 '北投區': 0.0,
 '北港鎮': 0.0,
 '北門區': 0.0,
 '卑南鄉': 0.0,
 '南州鄉': 0.0,
 '南投市': 0.0,
 '口湖鄉': 2336.3716097560978,
 '吉安鄉': 0.0,
 '和美鎮': 0.0,
 '員林市': 0.0,
 '四湖鄉': 0.0,
 '土城區': 0.0,
 '土庫鎮': 0.0,
 '埔里鎮': 0.0,
 '埔鹽鄉': 0.0,
 '壽豐鄉': 0.0,
 '大內區': 0.0,
 '大城鄉': 0.0,
 '大埤鄉': 0.0,
 '大安區': 0.0,
 '大寮區': 0.0,
 '大村鄉': 0.0,
 '大林鎮': 0.0,
 '大里區': 0.0,
 '太保市': 885.4282902208201,
 '安南區': 0.0,
 '宜蘭市': 10637.208266666667,
 '岡山區': 0.0,
 '崙背鄉': 0.0,
 '左鎮區': 0.0,
 '布袋鎮': 879.8770156739812,
 '平溪區': 0.0,
 '彰化市': 0.0,
 '後壁區': 0.0,
 '恆春鎮': 0.0,
 '文山區': 0.0,
 '斗南鎮': 71477.25255045872,
 '新化區': 0.0,
 '新埔鎮': 0.0,
 '新市區': 0.0,
 '新店區': 0.0,
 '新港鄉': 896.7436677316293,
 '新營區': 3988.4870515463913,
 '新莊區': 0.0,
 '旗山區': 3911.6611463414633,

In [307]:
save_categorized_df = categorized_df.rename(columns={'datetime_x': 'datetime', 'total_damage': 'total_flood_loss'})
columns_to_keep = ['datetime', 'TOWNNAME', 'COUNTYNAME', 'total_flood_loss']
save_categorized_df = save_categorized_df[columns_to_keep]
save_categorized_df['datetime'] = pd.to_datetime(save_categorized_df['datetime'])
save_categorized_df['year'] = save_categorized_df['datetime'].dt.year
yearly_dfs = {}
for year in range(2020, 2026):  # 2020-2025年
    yearly_df = save_categorized_df[save_categorized_df['year'] == year].copy()
    yearly_df = yearly_df.drop(columns=['year'])
    yearly_df.to_csv(os.path.join(os.path.join("..", 'output'), f'town_flood_loss_statistics_{year}.csv'), index=False)

In [271]:
tmp = pd.read_csv(os.path.join("..", "output", "town_stat.csv"))
pd.to_datetime(tmp['datetime_x'])

0        2020-01-02
1        2020-01-02
2        2020-01-02
3        2020-01-02
4        2020-01-02
            ...    
143350   2025-02-28
143351   2025-02-28
143352   2025-02-28
143353   2025-02-28
143354   2025-02-28
Name: datetime_x, Length: 143355, dtype: datetime64[ns]

In [265]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

nonzero_categorized_df = categorized_df[categorized_df['damage_class'] != 0]

# 以 rainfall 預測 flood_depth
X_lin = nonzero_categorized_df[['rainfall']]
y_lin = nonzero_categorized_df['flood_depth']

lin_model = LinearRegression()
lin_model.fit(X_lin, y_lin)

print("\n=== Linear Regression (Flood Depth given Flood) ===")
print("Intercept (bias):", lin_model.intercept_)
print("Coefficient for rainfall:", lin_model.coef_[0])

# 也可以產生預測淹水深度
nonzero_categorized_df['flood_depth_pred'] = lin_model.predict(X_lin)

from sklearn.metrics import r2_score

# 對僅有淹水 (flood_depth > 0) 的資料進行預測
y_lin_pred = lin_model.predict(X_lin)
r2 = r2_score(y_lin, y_lin_pred)
print("R^2 for flood depth linear regression:", r2)



=== Linear Regression (Flood Depth given Flood) ===
Intercept (bias): 30.641082392848038
Coefficient for rainfall: 0.030286274044261072
R^2 for flood depth linear regression: 0.0013717682978745538


/var/folders/8f/v52bhkrx7zldnfw3mqgwj1qc0000gn/T/ipykernel_87916/2354315143.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonzero_categorized_df['flood_depth_pred'] = lin_model.predict(X_lin)
